# Powerpoint Slides Summarizer

This converts a Power Point presentation into notes that a student can easily skim through.

Concepts Used:
- Converting Contents of PPT to text via python-pptx
- User and System Prompts
- Use of Open AI GPT-4o-mini via API key


In [0]:
!pip install python-pptx

In [0]:
# imports

import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from openai import OpenAI

In [0]:
# Load environment variables in a file called .env

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

# Check the key

if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif not api_key.startswith("sk-proj-"):
    print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook")
elif api_key.strip() != api_key:
    print("An API key was found, but it looks like it might have space or tab characters at the start or end - please remove them - see troubleshooting notebook")
else:
    print("API key found and looks good so far!")

In [0]:
# creating an instance
openai = OpenAI()

In [0]:
from pptx import Presentation

class PowerPoint():
    def __init__(self,ppt):
        """
        Creates a PowerPoint object, with name and text.
        """
        self.ppt = ppt
        self.title = os.path.basename(ppt)
        self.text = self.extract_text()

    def extract_text(self):
        """
        Extracts text from powerpoint.
        """
        prs = Presentation(self.ppt)
        text_content = []
    
        for slide in prs.slides:
            for shape in slide.shapes:
                if hasattr(shape, "text"):
                    text_content.append(shape.text)
    
        return "\n".join(text_content)
    

In [0]:
system_prompt = "You are an assistant that analyzes the contents \
of a PowerPoint presentation, and provides a summary in the style of \
a cheat-sheet, for students to easily learn key concepts from.\
You are to ignore text that might be navigation-related\
and respond in Markdown."

In [0]:
# A function that writes a User Prompt that asks for summaries of PowerPoints:

def user_prompt_for(powerpoint):
    user_prompt = f"You are looking at a website titled {powerpoint.title}"
    user_prompt += "\nThe contents of this powerpoint are as follows; \
please provide a summary of the content in markdown. \
If it includes a question bank, add that along with short answers too.\n\n"
    user_prompt += powerpoint.text
    return user_prompt

In [0]:
def messages_for(powerpoint):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(powerpoint)}
    ]

In [0]:
# And now: call the OpenAI API. 

def summarize(powerpoint_path):
    powerpoint = PowerPoint(powerpoint_path)
    response = openai.chat.completions.create(
        model = "gpt-4o-mini",
        messages = messages_for(powerpoint)
    )
    return response.choices[0].message.content

In [0]:
def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

In [0]:
ppt_file = "Theoretical Perspectives on Media and Technology.pptx" 
display_summary(ppt_file)